# Protein Data Structure I/O

通常情况下，我们可以从PDB文件中获取蛋白质的结构信息。PDB文件是一种标准的数据格式，用于描述蛋白质的结构。在本教程中，我们以单链胰岛素（PDB id: 2LWZ）作为示例。让我们首先使用NGLView对其进行可视化。

In [2]:
import nglview

view = nglview.show_pdbid("2lwz")
view

NGLWidget()

## Construct Protein Data Structure from PDB File

在TorchProtein中，我们可以使用Protein.from_pdb函数读取PDB文件并构建数据结构。原子(atom)、边(edge)和残基(residue)的特征可以作为机器学习模型的输入。我们可以通过更改Protein.from_pdb函数的参数来指定不同的特征。

In [3]:
import torchdrug as td
from torchdrug import data, utils

pdb_file = utils.download("https://files.rcsb.org/download/2LWZ.pdb", "./")
protein = data.Protein.from_pdb(pdb_file, atom_feature="position", bond_feature="length", residue_feature="symbol")
print(protein)
print(protein.residue_feature.shape)
print(protein.atom_feature.shape)
print(protein.bond_feature.shape)

19:07:14   Downloading https://files.rcsb.org/download/2LWZ.pdb to ./2LWZ.pdb
Protein(num_atom=445, num_bond=916, num_residue=57)
torch.Size([57, 21])
torch.Size([445, 3])
torch.Size([916, 1])


构建的数据结构包含了关于蛋白质的丰富信息。例如，你可以获取前10个残基的链ID和前10个原子的三维坐标，如下所示。

In [4]:
for residue_id, chain_id in zip(protein.residue_type.tolist()[:10], protein.chain_id.tolist()[:10]):
    print("%s: %s" % (data.Protein.id2residue[residue_id], chain_id))

for atom, position in zip(protein.atom_name.tolist()[:10], protein.node_position.tolist()[:10]):
    print("%s: %s" % (data.Protein.id2atom_name[atom], position))

PHE: 1
VAL: 1
ASN: 1
GLN: 1
HIS: 1
LEU: 1
CYS: 1
GLY: 1
SER: 1
ASP: 1
N: [-5.2789998054504395, -12.38599967956543, 2.930999994277954]
CA: [-5.0970001220703125, -11.57699966430664, 1.6929999589920044]
C: [-6.017000198364258, -12.133000373840332, 0.6019999980926514]
O: [-5.697999954223633, -12.098999977111816, -0.5690000057220459]
CB: [-3.632999897003174, -11.685999870300293, 1.2400000095367432]
CG: [-3.2190001010894775, -10.597999572753906, 0.27300000190734863]
CD1: [-4.142000198364258, -9.737000465393066, -0.3370000123977661]
CD2: [-1.8589999675750732, -10.461999893188477, -0.013000000268220901]
CE1: [-3.694999933242798, -8.76200008392334, -1.218000054359436]
CE2: [-1.4170000553131104, -9.482999801635742, -0.8930000066757202]


蛋白质的数据结构存储了恢复蛋白质所需的所有信息，并提供了一个to_pdb()方法，用于将蛋白质以PDB格式保存。我们以下面的方式展示了单链胰岛素的恢复过程。

In [5]:
from rdkit import Chem

protein.to_pdb("new_2LWZ.pdb")
mol = Chem.MolFromPDBFile("new_2LWZ.pdb")
view = nglview.show_rdkit(mol)
view

NGLWidget()

## Construct Protein Data Structure from Protein Sequence

在一些应用中，我们可能只能访问蛋白质的氨基酸序列。对于这种情况，TorchProtein提供了`Protein.from_sequence`方法和`Protein.from_sequence_fast`方法，用于从序列构建蛋白质的数据结构。前者使用RDKit构建蛋白质对象，它会计算原子、残基和键的特征，因此速度较慢。后者直接使用残基类型和特征构建蛋白质的数据结构，因此速度更快。

In [6]:
import time

aa_seq = protein.to_sequence()
print(aa_seq)

start_time = time.time()
seq_protein = data.Protein.from_sequence(aa_seq, atom_feature="symbol", bond_feature="length", residue_feature="symbol")
end_time = time.time()
print("Duration of construction: ", end_time - start_time)
print(seq_protein)

start_time = time.time()
seq_protein = data.Protein.from_sequence(aa_seq, atom_feature=None, bond_feature=None, residue_feature="default")
end_time = time.time()
print("Duration of construction: ", end_time - start_time)
print(seq_protein)

FVNQHLCGSDLVEALYLVCGERGFFYTDPTGGGPRRGIVEQCCHSICSLYQLENYCN
Duration of construction:  0.7388813495635986
Protein(num_atom=445, num_bond=910, num_residue=57)
Duration of construction:  0.001672983169555664
Protein(num_atom=0, num_bond=0, num_residue=57)


# Protein Operations

## Batch Protein

为了充分利用硬件资源，TorchProtein继承了TorchDrug中的data.Graph结构，并支持将多个蛋白质作为一个批次进行处理。批次可以通过cpu()和cuda()方法在CPU和GPU之间切换。给定多个蛋白质，我们可以通过data.Protein.pack方法构建蛋白质批次，并通过cuda()方法将其从CPU转移到GPU上。此外，我们还可以通过常规的索引操作从批次中提取出几个特定的蛋白质。

In [7]:
proteins = [protein] * 3
proteins = data.Protein.pack(proteins)
print(proteins)
proteins = proteins.cuda()
print(proteins)
proteins_ = proteins[[0, 2]]
print(proteins_)

PackedProtein(batch_size=3, num_atoms=[445, 445, 445], num_bonds=[916, 916, 916], num_residues=[57, 57, 57])
PackedProtein(batch_size=3, num_atoms=[445, 445, 445], num_bonds=[916, 916, 916], num_residues=[57, 57, 57], device='cuda:0')
PackedProtein(batch_size=2, num_atoms=[445, 445], num_bonds=[916, 916], num_residues=[57, 57], device='cuda:0')


## References between Atoms and Residues

在TorchProtein中，我们提供了atom2residue方法来获取每个原子对应的残基，同时提供了residue2atom方法来获取每个残基关联的原子。这两个方法的典型用法如下所示。

In [8]:
for atom_id, (atom, residue_id) in enumerate(zip(protein.atom_name.tolist()[:20], protein.atom2residue.tolist()[:20])):
    print("[atom %s] %s: %s" % (atom_id, data.Protein.id2atom_name[atom], data.Protein.id2residue[residue_id]))

for residue_id in [0, 1]:
    atom_ids = protein.residue2atom(residue_id).sort()[0]
    for atom, position in zip(protein.atom_name[atom_ids].tolist(), protein.node_position[atom_ids].tolist()):
        print("[residue %s] %s: %s" % (residue_id, data.Protein.id2atom_name[atom], position))

[atom 0] N: GLY
[atom 1] CA: GLY
[atom 2] C: GLY
[atom 3] O: GLY
[atom 4] CB: GLY
[atom 5] CG: GLY
[atom 6] CD1: GLY
[atom 7] CD2: GLY
[atom 8] CE1: GLY
[atom 9] CE2: GLY
[atom 10] CZ: GLY
[atom 11] N: ALA
[atom 12] CA: ALA
[atom 13] C: ALA
[atom 14] O: ALA
[atom 15] CB: ALA
[atom 16] CG1: ALA
[atom 17] CG2: ALA
[atom 18] N: SER
[atom 19] CA: SER
[residue 0] N: [-5.2789998054504395, -12.38599967956543, 2.930999994277954]
[residue 0] CA: [-5.0970001220703125, -11.57699966430664, 1.6929999589920044]
[residue 0] C: [-6.017000198364258, -12.133000373840332, 0.6019999980926514]
[residue 0] O: [-5.697999954223633, -12.098999977111816, -0.5690000057220459]
[residue 0] CB: [-3.632999897003174, -11.685999870300293, 1.2400000095367432]
[residue 0] CG: [-3.2190001010894775, -10.597999572753906, 0.27300000190734863]
[residue 0] CD1: [-4.142000198364258, -9.737000465393066, -0.3370000123977661]
[residue 0] CD2: [-1.8589999675750732, -10.461999893188477, -0.013000000268220901]
[residue 0] CE1: [-3.6

## Subprotein and Masking

在蛋白质研究中，我们有时需要从蛋白质中提取特定的残基并进行分析。使用TorchProtein，我们可以通过索引操作轻松实现这一点。我们以从蛋白质中提取前两个残基为例进行说明。请注意，在提取过程中，提取的残基之间的原子间的键将被保留。

In [9]:
first_two = protein[:2]
first_two.visualize()

在TorchProtein中，我们还提供了residue_mask方法来从蛋白质中提取指定的残基，并提供了node_mask方法来从蛋白质中提取指定的原子。通过使用这两个方法，我们也可以从蛋白质中提取前两个残基，具体操作如下所示。

In [10]:
is_first_two_ = (protein.residue_number == 1) | (protein.residue_number == 2)
first_two_ = protein.residue_mask(is_first_two_, compact=True)
assert first_two == first_two_

is_first_two_ = (protein.atom2residue == 0) | (protein.atom2residue == 1)
first_two_ = protein.node_mask(is_first_two_, compact=True)
assert first_two == first_two_

## Atom and Residue Views

对于基于序列的蛋白质编码模型，我们通常将残基视为蛋白质图中的节点，而有时我们也希望将原子特征作为结构基础的蛋白质编码模型的节点特征。为了支持在原子和残基特征之间灵活切换，TorchProtein定义了view属性，用于选择我们想要用作节点特征的特征类型。

In [11]:
protein.view = "atom"
print(protein.node_feature.shape)
protein.view = "residue"
print(protein.node_feature.shape)

torch.Size([445, 3])
torch.Size([57, 21])


# Register Your Own Attributes

虽然Protein类带有一些原子级和残基级的属性，但我们可能还想定义自己的属性。这只需要将属性赋值行包装在上下文管理器中即可。我们可以分别使用protein.atom()、protein.residue()和protein.graph()来处理原子级、残基级和图级属性。

## Register Residue and Atom Attributes

我们在这里给出了两个注册残基和原子属性的示例。第一个示例定义了一个自定义的残基属性，用于编码每个残基是否后跟一个“GLY”残基。第二个示例定义了一个自定义的原子属性，用于编码每个原子是否连接到一个氮原子。

In [13]:
import torch
from torch_scatter import scatter_add

next_residue_type = torch.cat([protein.residue_type[1:], torch.full((1,), -1, dtype=protein.residue_type.dtype)])
followed_by_GLY = next_residue_type == data.Protein.residue2id["GLY"]
with protein.residue():
    protein.followed_by_GLY = followed_by_GLY

atom_in, atom_out = protein.edge_list.t()[:2]
attached_to_N = scatter_add(protein.atom_type[atom_in] == td.NITROGEN, atom_out, dim_size=protein.num_node)
with protein.atom():
    protein.attached_to_N = attached_to_N

## Register References between Residues and Atoms

在某些情况下，我们希望将一个残基/原子链接到另一个残基/原子。我们可以通过在protein.residue_reference()或protein.atom_reference()的上下文中进行注册来实现这一点。例如，我们可以在protein.residue()和protein.atom_reference()的上下文中注册每个残基对应的α-碳的索引。请注意，在提取蛋白质的一部分的任何操作下，以这种方式注册的索引将自动更改为新提取蛋白质下的索引。

In [14]:
from torch_scatter import scatter_max

range = torch.arange(protein.num_node)
calpha = torch.where(protein.atom_name == protein.atom_name2id["CA"], range, -1)
residue2calpha = scatter_max(calpha, protein.atom2residue, dim_size=protein.num_residue)[0]
with protein.residue(), protein.atom_reference():
    protein.residue2calpha = residue2calpha

sub_protein = protein[3:10]
for calpha_index in sub_protein.residue2calpha.tolist():
    atom_name = data.Protein.id2atom_name[sub_protein.atom_name[calpha_index].item()]
    print("New index %d: %s" % (calpha_index, atom_name))

New index 1: CA
New index 10: CA
New index 20: CA
New index 28: CA
New index 34: CA
New index 38: CA
New index 44: CA
